In [16]:
import cProfile
import pstats
from pstats import SortKey
import pandas as pd

In [38]:
#to increase float precision a bit
def f8_alt(x):
    return "%14.5f" % x
pstats.f8 = f8_alt

In [39]:
#stats for 16 concurrent calls
p16 = pstats.Stats('../log/deepCoadd_calexp_transfer_16.prof')

In [40]:
p16.strip_dirs().sort_stats("cumulative")
p16.print_stats(50)

Thu Feb 22 19:52:52 2024    ../log/deepCoadd_calexp_transfer_16.prof

         1935134104 function calls (1935086278 primitive calls) in 6525.986 seconds

   Ordered by: cumulative time
   List reduced from 730 to 50 due to restriction <50>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1        0.00003        0.00003     6526.00803     6526.00803 base_events.py:617(run_until_complete)
        1       11.04105       11.04105     6526.00794     6526.00794 base_events.py:593(run_forever)
 13620978      143.66989        0.00001     6514.96686        0.00048 base_events.py:1845(_run_once)
 42387469       49.40354        0.00000     4855.56001        0.00011 events.py:78(_run)
 42387469      104.59152        0.00000     4806.15647        0.00011 {method 'run' of '_contextvars.Context' objects}
 19367117       17.74309        0.00000     3591.14550        0.00019 selector_events.py:951(_read_ready)
 19367117       62.26250        0.00000     3573.40242      

In [41]:
p16_stats = p16.get_stats_profile()

In [42]:
def stats_to_dict(stats):
    func_stat_dir = {
        'fname' : [],
        'ncalls' : [],
        'tottime' : [],
        'percall_tottime' : [],
        'cumtime': [],
        'percall_cumtime':[],
        'file_name':[],
        'line_number':[]
    }
    
    keys  = func_stat_dir.keys()
    # c=0
    
    for k,v in stats.func_profiles.items():
        func_stat_dir['fname'].append(k)
        func_attrs = dir(v)
        for attr in func_attrs:
            if attr in keys:
                func_stat_dir[attr].append(getattr(v,attr))

    return func_stat_dir


In [43]:
#transform stats to dictionary so we can build a dataframe, it's easier to manipulate
p16_stats_dict = stats_to_dict(p16_stats)

In [44]:
#build df
p16_df = pd.DataFrame.from_dict(p16_stats_dict)

In [46]:
p16_df.to_csv("../log/deepCoadd_calexp_transfer_stats.csv",index=False)

In [55]:
p16_df[p16_df.file_name == 'async_http_transfer.py']

,fname,ncalls,tottime,percall_tottime,cumtime,percall_cumtime,file_name,line_number
14,download_and_write,16810235,9.70545,0.00000,1050.86828,0.00006,async_http_transfer.py,23
15,fetch,16775183,8.46006,0.00000,1034.88662,0.00006,async_http_transfer.py,12
67,write_fits_file,46736,0.24888,0.00001,5.03122,0.00011,async_http_transfer.py,16
160,main,3,0.00039,0.00013,0.19089,0.06363,async_http_transfer.py,39


In [150]:
p16_df.head(10)

,fname,ncalls,tottime,percall_tottime,cumtime,percall_cumtime,file_name,line_number
0,run_until_complete,1,0.00003,0.00003,6526.00803,6526.00803,base_events.py,617
1,run_forever,1,11.04105,11.04105,6526.00794,6526.00794,base_events.py,593
2,_run_once,13620978,143.66989,0.00001,6514.96686,0.00048,base_events.py,1845
3,_run,42387469,49.40354,0.00000,4855.56001,0.00011,events.py,78
4,<method 'run' of '_contextvars.Context' objects>,42387469,104.59152,0.00000,4806.15647,0.00011,~,0
5,_read_ready,19367117,17.74309,0.00000,3591.14550,0.00019,selector_events.py,951
6,_read_ready__get_buffer,19367117,62.26250,0.00000,3573.40242,0.00018,selector_events.py,954
7,buffer_updated,19367117,61.52892,0.00000,2917.51725,0.00015,sslproto.py,432
8,_do_read,25414289,70.55235,0.00000,2736.32404,0.00011,sslproto.py,720
9,_do_read__copied,21542096,175.63982,0.00001,2543.62760,0.00012,sslproto.py,771


In [74]:
p16_df[p16_df.fname=="main"]

,fname,ncalls,tottime,percall_tottime,cumtime,percall_cumtime,file_name,line_number
160,main,3,0.00039,0.00013,0.19089,0.06363,async_http_transfer.py,39


In [90]:
#stats for 32 concurrent calls
p32 = pstats.Stats('../log/deepCoadd_calexp_transfer_32.prof')
p32.strip_dirs().sort_stats("cumulative")
p32_stats = p32.get_stats_profile()
p32.print_stats()

Fri Feb 23 13:19:48 2024    ../log/deepCoadd_calexp_transfer_32.prof

         1348478128 function calls (1348430275 primitive calls) in 4565.049 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1        0.00003        0.00003     4565.07755     4565.07755 base_events.py:617(run_until_complete)
        1        2.74751        2.74751     4565.07745     4565.07745 base_events.py:593(run_forever)
  2249657       50.28866        0.00002     4562.32991        0.00203 base_events.py:1845(_run_once)
 29192880       28.75335        0.00000     4365.34411        0.00015 events.py:78(_run)
 29192880      138.56676        0.00000     4336.59076        0.00015 {method 'run' of '_contextvars.Context' objects}
 11019504       10.16245        0.00000     2972.29872        0.00027 selector_events.py:951(_read_ready)
 11019504       35.13081        0.00000     2962.13627        0.00027 selector_events.py:954(_read_ready__get_buffe

In [87]:
#transform stats to dictionary so we can build a dataframe, it's easier to manipulate
p32_stats_dict = stats_to_dict(p32_stats)
p32_df = pd.DataFrame.from_dict(p32_stats_dict)
p32_df.to_csv("../log/deepCoadd_calexp_transfer_stats_32.csv",index=False)

In [151]:
p32_df.head(10)

,fname,ncalls,tottime,percall_tottime,cumtime,percall_cumtime,file_name,line_number
0,run_until_complete,1,0.00003,0.00003,4565.07755,4565.07755,base_events.py,617
1,run_forever,1,2.74751,2.74751,4565.07745,4565.07745,base_events.py,593
2,_run_once,2249657,50.28866,0.00002,4562.32991,0.00203,base_events.py,1845
3,_run,29192880,28.75335,0.00000,4365.34411,0.00015,events.py,78
4,<method 'run' of '_contextvars.Context' objects>,29192880,138.56676,0.00000,4336.59076,0.00015,~,0
5,_read_ready,11019504,10.16245,0.00000,2972.29872,0.00027,selector_events.py,951
6,_read_ready__get_buffer,11019504,35.13081,0.00000,2962.13627,0.00027,selector_events.py,954
7,buffer_updated,11019504,33.51167,0.00000,2443.07954,0.00022,sslproto.py,432
8,_do_read,18375534,42.35978,0.00000,2262.61227,0.00012,sslproto.py,720
9,_do_read__copied,12722373,130.46891,0.00001,2145.05113,0.00017,sslproto.py,771


In [88]:
p32_df[p32_df.file_name == 'async_http_transfer.py']

,fname,ncalls,tottime,percall_tottime,cumtime,percall_cumtime,file_name,line_number
12,download_and_write,10656869,6.57763,0.00000,1175.26859,0.00011,async_http_transfer.py,23
13,fetch,10621817,5.66031,0.00000,1156.68852,0.00011,async_http_transfer.py,12
63,write_fits_file,46736,0.26927,0.00001,5.24683,0.00011,async_http_transfer.py,16
171,main,3,0.00034,0.00011,0.16900,0.05633,async_http_transfer.py,39


In [91]:
#stats for 64 concurrent calls
p64 = pstats.Stats('../log/deepCoadd_calexp_transfer_64.prof')
p64.strip_dirs().sort_stats("cumulative")
p64_stats = p64.get_stats_profile()
p64.print_stats()

Fri Feb 23 15:48:06 2024    ../log/deepCoadd_calexp_transfer_64.prof

         1199512151 function calls (1199463867 primitive calls) in 4447.565 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1        0.00003        0.00003     4447.59363     4447.59363 base_events.py:617(run_until_complete)
        1        0.50563        0.50563     4447.59353     4447.59353 base_events.py:593(run_forever)
   207488       35.24118        0.00017     4447.08787        0.02143 base_events.py:1845(_run_once)
 25619986       27.10322        0.00000     4365.19832        0.00017 events.py:78(_run)
 25619986      148.35402        0.00001     4338.09510        0.00017 {method 'run' of '_contextvars.Context' objects}
  8846526        9.74897        0.00000     2937.85725        0.00033 selector_events.py:951(_read_ready)
  8846526       29.82865        0.00000     2928.10828        0.00033 selector_events.py:954(_read_ready__get_buffe

In [92]:
#transform stats to dictionary so we can build a dataframe, it's easier to manipulate
p64_stats_dict = stats_to_dict(p64_stats)
p64_df = pd.DataFrame.from_dict(p64_stats_dict)
p64_df.to_csv("../log/deepCoadd_calexp_transfer_stats_64.csv",index=False)

In [93]:
p64_df[p64_df.file_name == 'async_http_transfer.py']

,fname,ncalls,tottime,percall_tottime,cumtime,percall_cumtime,file_name,line_number
12,download_and_write,8863173,6.66231,0.00000,1201.80117,0.00014,async_http_transfer.py,23
13,fetch,8828118,5.77837,0.00000,1187.84245,0.00013,async_http_transfer.py,12
57,write_fits_file,46740,0.30318,0.00001,5.91870,0.00013,async_http_transfer.py,16
151,main,3,0.00038,0.00013,0.21761,0.07254,async_http_transfer.py,39


In [94]:
#stats for 128 concurrent calls
p128 = pstats.Stats('../log/deepCoadd_calexp_transfer_128.prof')
p128.strip_dirs().sort_stats("cumulative")
p128_stats = p128.get_stats_profile()
p128.print_stats()

Fri Feb 23 17:18:36 2024    ../log/deepCoadd_calexp_transfer_128.prof

         1137487432 function calls (1137439041 primitive calls) in 4263.167 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1        0.00003        0.00003     4263.19713     4263.19713 base_events.py:617(run_until_complete)
        1        0.22912        0.22912     4263.19703     4263.19703 base_events.py:593(run_forever)
    69994       35.46064        0.00051     4262.96789        0.06090 base_events.py:1845(_run_once)
 24061291       28.76072        0.00000     4185.51166        0.00017 events.py:78(_run)
 24061291      122.82107        0.00001     4156.75093        0.00017 {method 'run' of '_contextvars.Context' objects}
  8006111        9.47601        0.00000     2906.69481        0.00036 selector_events.py:951(_read_ready)
  8006111       32.22028        0.00000     2897.21880        0.00036 selector_events.py:954(_read_ready__get_buff

In [95]:
#transform stats to dictionary so we can build a dataframe, it's easier to manipulate
p128_stats_dict = stats_to_dict(p128_stats)
p128_df = pd.DataFrame.from_dict(p128_stats_dict)
p128_df.to_csv("../log/deepCoadd_calexp_transfer_stats_128.csv",index=False)

In [96]:
p128_df[p128_df.file_name == 'async_http_transfer.py']

,fname,ncalls,tottime,percall_tottime,cumtime,percall_cumtime,file_name,line_number
12,download_and_write,8065303,6.25942,0.00000,1077.86141,0.00013,async_http_transfer.py,23
13,fetch,8030248,5.45163,0.00000,1058.27951,0.00013,async_http_transfer.py,12
58,write_fits_file,46740,0.30139,0.00001,5.51081,0.00012,async_http_transfer.py,16
159,main,3,0.00034,0.00011,0.17847,0.05949,async_http_transfer.py,39


In [135]:
p128_df.sort_values('tottime',ascending=False).head(20)

,fname,ncalls,tottime,percall_tottime,cumtime,percall_cumtime,file_name,line_number
11,<method 'read' of '_ssl._SSLSocket' objects>,132848121,1216.96477,0.00001,1216.96477,0.00001,~,0
14,<method 'join' of 'bytes' objects>,16014123,978.51363,0.00006,978.51363,0.00006,~,0
15,<method 'recv_into' of '_socket.socket' objects>,8006111,499.10448,0.00006,499.10448,0.00006,~,0
17,<method 'write' of '_ssl.MemoryBIO' objects>,8006111,296.54896,0.00004,296.54896,0.00004,~,0
16,data_received,8002143,228.91059,0.00003,378.61028,0.00005,client_proto.py,201
4,<method 'run' of '_contextvars.Context' objects>,24061291,122.82107,0.00001,4156.75093,0.00017,~,0
9,_do_read__copied,8539511,112.94270,0.00001,1947.31089,0.00023,sslproto.py,771
8,_do_read,15841162,35.86947,0.00000,2055.03585,0.00013,sslproto.py,720
2,_run_once,69994,35.46064,0.00051,4262.96789,0.06090,base_events.py,1845
6,_read_ready__get_buffer,8006111,32.22028,0.00000,2897.21880,0.00036,selector_events.py,954


In [148]:
p128.print_callers("method 'read' of")

   Ordered by: cumulative time
   List reduced from 730 to 3 due to restriction <"method 'read' of">

Function                                         was called by...
                                                     ncalls  tottime  cumtime
{method 'read' of '_ssl._SSLSocket' objects}     <- 132848121     1216.96477     1216.96477  ssl.py:912(read)
{method 'read' of '_ssl.MemoryBIO' objects}      <-    1507        0.00187        0.00187  sslproto.py:387(connection_lost)
                                                    8555945       11.80484       11.80484  sslproto.py:711(_process_outgoing)
{method 'read' of '_io.BufferedReader' objects}  <-       1        0.00094        0.00094  <frozen importlib._bootstrap_external>:1127(get_data)




In [168]:
most_cumtime_128 = p128_df.head(10)[['fname','cumtime']]

In [169]:
most_cumtime_128

,fname,cumtime
0,run_until_complete,4263.19713
1,run_forever,4263.19703
2,_run_once,4262.96789
3,_run,4185.51166
4,<method 'run' of '_contextvars.Context' objects>,4156.75093
5,_read_ready,2906.69481
6,_read_ready__get_buffer,2897.21880
7,buffer_updated,2350.27754
8,_do_read,2055.03585
9,_do_read__copied,1947.31089


In [170]:
most_cumtime_64 = p64_df.head(10)[['fname','cumtime']]
most_cumtime_64

,fname,cumtime
0,run_until_complete,4447.59363
1,run_forever,4447.59353
2,_run_once,4447.08787
3,_run,4365.19832
4,<method 'run' of '_contextvars.Context' objects>,4338.09510
5,_read_ready,2937.85725
6,_read_ready__get_buffer,2928.10828
7,buffer_updated,2409.49175
8,_do_read,2146.58804
9,_do_read__copied,2036.91994


In [171]:
most_cumtime_32 = p32_df.head(10)[['fname','cumtime']]
most_cumtime_32

,fname,cumtime
0,run_until_complete,4565.07755
1,run_forever,4565.07745
2,_run_once,4562.32991
3,_run,4365.34411
4,<method 'run' of '_contextvars.Context' objects>,4336.59076
5,_read_ready,2972.29872
6,_read_ready__get_buffer,2962.13627
7,buffer_updated,2443.07954
8,_do_read,2262.61227
9,_do_read__copied,2145.05113


In [172]:
most_cumtime_16 = p16_df.head(10)[['fname','cumtime']]
most_cumtime_16

,fname,cumtime
0,run_until_complete,6526.00803
1,run_forever,6526.00794
2,_run_once,6514.96686
3,_run,4855.56001
4,<method 'run' of '_contextvars.Context' objects>,4806.15647
5,_read_ready,3591.14550
6,_read_ready__get_buffer,3573.40242
7,buffer_updated,2917.51725
8,_do_read,2736.32404
9,_do_read__copied,2543.62760


In [176]:
most_cumtime = most_cumtime_16

In [184]:
most_cumtime = most_cumtime.rename(columns={'cumtime':'cumtime_16'})

In [188]:
most_cumtime['cumtime_32']=most_cumtime_32['cumtime']

In [190]:
most_cumtime['cumtime_64']=most_cumtime_64['cumtime']
most_cumtime['cumtime_128']=most_cumtime_128['cumtime']

In [191]:
most_cumtime

,fname,cumtime_16,cumtime_32,cumtime_64,cumtime_128
0,run_until_complete,6526.00803,4565.07755,4447.59363,4263.19713
1,run_forever,6526.00794,4565.07745,4447.59353,4263.19703
2,_run_once,6514.96686,4562.32991,4447.08787,4262.96789
3,_run,4855.56001,4365.34411,4365.19832,4185.51166
4,<method 'run' of '_contextvars.Context' objects>,4806.15647,4336.59076,4338.09510,4156.75093
5,_read_ready,3591.14550,2972.29872,2937.85725,2906.69481
6,_read_ready__get_buffer,3573.40242,2962.13627,2928.10828,2897.21880
7,buffer_updated,2917.51725,2443.07954,2409.49175,2350.27754
8,_do_read,2736.32404,2262.61227,2146.58804,2055.03585
9,_do_read__copied,2543.62760,2145.05113,2036.91994,1947.31089


In [192]:
most_cumtime.to_csv("../log/deepCoadd_calexp_transfer_stats_cumtime.csv",index=False)

In [227]:
most_tottime_128=p128_df.sort_values(by='tottime',ascending=False)
most_tottime_128=most_tottime_128.head(20)[['fname','tottime']]

In [228]:
most_tottime_128

,fname,tottime
11,<method 'read' of '_ssl._SSLSocket' objects>,1216.96477
14,<method 'join' of 'bytes' objects>,978.51363
15,<method 'recv_into' of '_socket.socket' objects>,499.10448
17,<method 'write' of '_ssl.MemoryBIO' objects>,296.54896
16,data_received,228.91059
4,<method 'run' of '_contextvars.Context' objects>,122.82107
9,_do_read__copied,112.94270
8,_do_read,35.86947
2,_run_once,35.46064
6,_read_ready__get_buffer,32.22028


In [229]:
most_tottime_64=p64_df.sort_values(by='tottime',ascending=False)
most_tottime_64=most_tottime_64.head(20)[['fname','tottime']]
# most_tottime_64=most_tottime_64[most_tottime_64.tottime>=60][['fname','tottime']]

In [230]:
most_tottime_64

,fname,tottime
11,<method 'read' of '_ssl._SSLSocket' objects>,1279.30473
14,<method 'join' of 'bytes' objects>,1125.72688
15,<method 'recv_into' of '_socket.socket' objects>,474.25158
17,<method 'write' of '_ssl.MemoryBIO' objects>,269.48436
16,data_received,227.37307
4,<method 'run' of '_contextvars.Context' objects>,148.35402
9,_do_read__copied,115.30074
8,_do_read,36.75064
2,_run_once,35.24118
6,_read_ready__get_buffer,29.82865


In [231]:
most_tottime_32=p32_df.sort_values(by='tottime',ascending=False)
# most_tottime_32=most_tottime_32[most_tottime_32.tottime>=60][['fname','tottime']]
most_tottime_32=most_tottime_32.head(20)[['fname','tottime']]
most_tottime_32

,fname,tottime
11,<method 'read' of '_ssl._SSLSocket' objects>,1333.23007
14,<method 'join' of 'bytes' objects>,1077.56603
15,<method 'recv_into' of '_socket.socket' objects>,468.73378
16,data_received,232.80432
18,<method 'write' of '_ssl.MemoryBIO' objects>,182.57457
4,<method 'run' of '_contextvars.Context' objects>,138.56676
9,_do_read__copied,130.46891
22,<method 'poll' of 'select.epoll' objects>,93.76550
2,_run_once,50.28866
8,_do_read,42.35978


In [232]:
most_tottime_16=p16_df.sort_values(by='tottime',ascending=False)
# most_tottime_16=most_tottime_16[most_tottime_16.tottime>=60][['fname','tottime']]
most_tottime_16=most_tottime_16.head(20)[['fname','tottime']]
most_tottime_16

,fname,tottime
11,<method 'read' of '_ssl._SSLSocket' objects>,1518.56725
13,<method 'poll' of 'select.epoll' objects>,1345.90008
16,<method 'join' of 'bytes' objects>,839.72411
17,<method 'recv_into' of '_socket.socket' objects>,567.27752
18,data_received,273.38329
9,_do_read__copied,175.63982
21,<method 'write' of '_ssl.MemoryBIO' objects>,164.59953
2,_run_once,143.66989
4,<method 'run' of '_contextvars.Context' objects>,104.59152
12,select,73.42633


In [249]:
tot_time_per_func = pd.DataFrame()

In [250]:
tot_time_per_func['fname'] = p16_df['fname']

In [251]:
tot_time_per_func['tottime_16'] = p16_df['tottime']

In [252]:
tot_time_per_func = tot_time_per_func.merge(p32_df[['fname','tottime']],on='fname')

In [253]:
tot_time_per_func = tot_time_per_func.rename(columns={'tottime':'tottime_32'})

In [254]:
tot_time_per_func

,fname,tottime_16,tottime_32
0,run_until_complete,0.00003,0.00003
1,run_forever,11.04105,2.74751
2,_run_once,143.66989,50.28866
3,_run,49.40354,28.75335
4,<method 'run' of '_contextvars.Context' objects>,104.59152,138.56676
...,...,...,...
549,get_filename,0.00000,0.00000
550,<method 'remove_done_callback' of '_asyncio.Ta...,0.00000,0.00000
551,<method 'release' of '_thread.RLock' objects>,0.00000,0.00000
552,_relax_case,0.00000,0.00000


In [255]:
tot_time_per_func = tot_time_per_func.merge(p64_df[['fname','tottime']],on='fname')
tot_time_per_func = tot_time_per_func.rename(columns={'tottime':'tottime_64'})

tot_time_per_func = tot_time_per_func.merge(p128_df[['fname','tottime']],on='fname')
tot_time_per_func = tot_time_per_func.rename(columns={'tottime':'tottime_128'})

In [256]:
tot_time_per_func

,fname,tottime_16,tottime_32,tottime_64,tottime_128
0,run_until_complete,0.00003,0.00003,0.00003,0.00003
1,run_forever,11.04105,2.74751,0.50563,0.22912
2,_run_once,143.66989,50.28866,35.24118,35.46064
3,_run,49.40354,28.75335,27.10322,28.76072
4,<method 'run' of '_contextvars.Context' objects>,104.59152,138.56676,148.35402,122.82107
...,...,...,...,...,...
549,get_filename,0.00000,0.00000,0.00000,0.00000
550,<method 'remove_done_callback' of '_asyncio.Ta...,0.00000,0.00000,0.00000,0.00000
551,<method 'release' of '_thread.RLock' objects>,0.00000,0.00000,0.00000,0.00000
552,_relax_case,0.00000,0.00000,0.00000,0.00000


In [279]:
import numpy as np
most_tot_time_fname = tot_time_per_func.sort_values('tottime_16',ascending=False).head(20).fname.values
most_tot_time_fname

array(["<method 'read' of '_ssl._SSLSocket' objects>",
       "<method 'poll' of 'select.epoll' objects>",
       "<method 'join' of 'bytes' objects>",
       "<method 'recv_into' of '_socket.socket' objects>",
       'data_received', '_do_read__copied',
       "<method 'write' of '_ssl.MemoryBIO' objects>", '_run_once',
       "<method 'run' of '_contextvars.Context' objects>", 'select',
       '_do_read', '_read_ready__get_buffer', 'buffer_updated', '_run',
       '_wait', '_read_nowait_chunk', '_call_soon', '_read_nowait',
       'readany', '_process_outgoing'], dtype=object)

In [280]:
most_tot_time_fname=np.append(
    most_tot_time_fname,
    tot_time_per_func.sort_values('tottime_32',ascending=False).head(20).fname.values
)

In [282]:
most_tot_time_fname=np.append(
    most_tot_time_fname,
    tot_time_per_func.sort_values('tottime_64',ascending=False).head(20).fname.values
)

In [283]:
most_tot_time_fname=np.append(
    most_tot_time_fname,
    tot_time_per_func.sort_values('tottime_128',ascending=False).head(20).fname.values
)

In [289]:
most_tot_time_fname_set = list(set(most_tot_time_fname))

In [290]:
len(most_tot_time_fname_set)

23

In [291]:
most_tot_time_per_func=tot_time_per_func[tot_time_per_func['fname'].isin(most_tot_time_fname_set)]
most_tot_time_per_func

,fname,tottime_16,tottime_32,tottime_64,tottime_128
2,_run_once,143.66989,50.28866,35.24118,35.46064
3,_run,49.40354,28.75335,27.10322,28.76072
4,<method 'run' of '_contextvars.Context' objects>,104.59152,138.56676,148.35402,122.82107
6,_read_ready__get_buffer,62.26250,35.13081,29.82865,32.22028
7,buffer_updated,61.52892,33.51167,26.86567,27.61424
8,_do_read,70.55235,42.35978,36.75064,35.86947
9,_do_read__copied,175.63982,130.46891,115.30074,112.94270
11,<method 'read' of '_ssl._SSLSocket' objects>,1518.56725,1333.23007,1279.30473,1216.96477
12,select,73.42633,20.60347,11.51966,9.20645
13,<method 'poll' of 'select.epoll' objects>,1345.90008,93.76550,13.03996,13.28506


In [292]:
most_tot_time_per_func.to_csv("../log/deepCoadd_calexp_transfer_stats_tottime.csv",index=False)

In [295]:
p128.print_stats("method 'read' of '_ssl._SSLSocket' objects")

Fri Feb 23 17:18:36 2024    ../log/deepCoadd_calexp_transfer_128.prof

         1137487432 function calls (1137439041 primitive calls) in 4263.167 seconds

   Ordered by: cumulative time
   List reduced from 730 to 1 due to restriction <"method 'read' of '_ssl._SSLSocket' objects">

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
132848121     1216.96477        0.00001     1216.96477        0.00001 {method 'read' of '_ssl._SSLSocket' objects}




In [296]:
#stats for 16 concurrent calls
p16 = pstats.Stats('../log/deepCoadd_calexp_transfer_16.prof')

In [297]:
p16.print_stats("method 'read' of '_ssl._SSLSocket' objects")

Thu Feb 22 19:52:52 2024    ../log/deepCoadd_calexp_transfer_16.prof

         1935134104 function calls (1935086278 primitive calls) in 6525.986 seconds

   Random listing order was used
   List reduced from 730 to 1 due to restriction <"method 'read' of '_ssl._SSLSocket' objects">

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
145877460     1518.56725        0.00001     1518.56725        0.00001 {method 'read' of '_ssl._SSLSocket' objects}




In [310]:
p16.print_callees("method 'read' of '_ssl._SSLSocket' objects")

   Random listing order was used
   List reduced from 730 to 1 due to restriction <"method 'read' of '_ssl._SSLSocket' objects">

Function                                      called...
                                                  ncalls  tottime  cumtime
{method 'read' of '_ssl._SSLSocket' objects}  -> 




In [304]:
p16.print_callers("method 'join' of 'bytes' objects")

   Random listing order was used
   List reduced from 730 to 1 due to restriction <"method 'join' of 'bytes' objects">

Function                            was called by...
                                        ncalls  tottime  cumtime
{method 'join' of 'bytes' objects}  <- 14092264      229.45058      229.45058  /home/ubuntu/lsst_stack_26/conda/envs/lsst-scipipe-7.0.1/lib/python3.11/asyncio/sslproto.py:771(_do_read__copied)
                                         11684      607.16089      607.16089  /home/ubuntu/lsst_stack_26/conda/envs/lsst-scipipe-7.0.1/lib/python3.11/site-packages/aiohttp/streams.py:345(read)
                                      16748651        3.11264        3.11264  /home/ubuntu/lsst_stack_26/conda/envs/lsst-scipipe-7.0.1/lib/python3.11/site-packages/aiohttp/streams.py:490(_read_nowait)




In [309]:
p16.print_callees("method 'recv_into' of '_socket.socket' objects")

   Random listing order was used
   List reduced from 730 to 1 due to restriction <"method 'recv_into' of '_socket.socket' objects">

Function                                          called...
                                                      ncalls  tottime  cumtime
{method 'recv_into' of '_socket.socket' objects}  -> 


